In [ ]:
import requests
import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os
import re
import pandas as pd
import csv
import time
from requests_html import HTML
from requests_html import HTMLSession
from itertools import cycle
import cloudscraper

sess = requests.session()
scraper = cloudscraper.create_scraper(sess)
header = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.81 Safari/537.36'
}


UseProxy = False
UseDelay = True
proxies = ['103.149.162.194:80', '38.94.111.208:80', '88.99.10.250:1080', '27.64.17.187:4203', '80.48.119.28:8080']
def filter_string(string):
    if '[' or ']' in string:
        string = string.replace('[', ' ')
        string = string.replace(']', ' ')
    string = string.replace("'", " ")
    string.replace('  ', ' ')
    string.replace('   ', ' ')
    string = string.strip()
    return string
def filter_html(string):
    check1 = 0
    check2 = 0
    while check1!=-1 and check2!=-1:
        if '<' and '>' in string:
            end = string.index('>')
            start = string.index('<')
            string = string.replace(string[start:end+1], ' ')
            check1 = string.find('<')
            check2 = string.find('>')
        else:
            break
    return string.strip()
def get_source(url):
    
    if UseProxy:
        done=False
        while done==False:
            
            proxy_pool = cycle(proxies)
            for p in range(1,11):
                #Get a proxy from the pool
                proxy = next(proxy_pool)
                
                try:
                    session = requests.session()
                    session = cloudscraper.create_scraper(sess = session)
                    session.proxies = {
                                "http": proxy,
                                "https": proxy,
                                }
                    response = session.get(url, headers=header)
                    response.raise_for_status() # if response is successfull, no exception will be raised
                    done=True
                    
                    return response
                    
                
                except requests.exceptions.RequestException as e:
                    print(e)
                    print(" ")
                    print("retrying with new proxy")
                    
                except requests.exceptions.HTTPError as err:
                    print(err)
                    print(" ")
                    print("retrying with new proxy")    
    else:
        try:
            session = requests.session()
            session = cloudscraper.create_scraper(sess = session)
            session.headers['user-agent'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.81 Safari/537.36'
            response = session.get(url)
            response.raise_for_status() # if response is successfull, no exception will be raised   
            return response
                            
        except requests.exceptions.RequestException as e:
            print(e)
                                
        except requests.exceptions.HTTPError as err:
            print(err)
            
        except cloudscraper.exceptions.CloudflareChallengeError as error:
            print(error)
            pass
            
def scrape_google(query):

     
    query = urllib.parse.quote_plus(query)
    
   
    response = get_source("https://www.google.com/search?q=" + query + "&num=100")
    soup = BeautifulSoup(response.content, 'html.parser')
    
    relative_keywords = soup.find('div', {'çlass':'y6Uyqe'})
    decks = soup.findAll('div', {'class':'AJLUJb'})
    searches = []
    for deck in decks:
        cards = deck.findAll('div')
        for card in cards:
            if card.text != '':
                searches.append(card.text)
    cards = soup.findAll('div', {'class' : 'yuRUbf'})
    links = []
    for card in cards:
        link = card.find('a')['href']
        links.append(link)
        
    google_domains = ('https://www.google.', 
                      'https://google.', 
                      'https://webcache.googleusercontent.', 
                      'http://webcache.googleusercontent.', 
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.',
                      'https://translate.google.',
                      'https://www.youtube')

    for url in links[:]:
        
        if url.startswith(google_domains):
            links.remove(url)
    
    
    return links, searches

def main():
    
    input_user = pd.read_csv('keyword_example.csv')
    c = len(input_user)
    i=0
    while i<c:
        output_data={
                'keyword' : [],
                'Top1 Link': [],
                'Top1 H1': [],
                'Top1 Paragraph after H1': [],
                'Top1 H2': [],
                'Top1 Paragraph after H2': [],
                'Top1 H3': [],
                'Top1 Paragraph after H3': [],
                'Top2 Link': [],
                'Top2 H1': [],
                'Top2 Paragraph after H1': [],
                'Top2 H2': [],
                'Top2 Paragraph after H2': [],
                'Top2 H3': [],
                'Top2 Paragraph after H3': [],
                'Top3 Link': [],
                'Top3 H1': [],
                'Top3 Paragraph after H1': [],
                'Top3 H2': [],
                'Top3 Paragraph after H2': [],
                'Top3 H3': [],
                'Top3 Paragraph after H3': [],
                'Top4 Link': [],
                'Top4 H1': [],
                'Top4 Paragraph after H1': [],
                'Top4 H2': [],
                'Top4 Paragraph after H2': [],
                'Top4 H3': [],
                'Top4 Paragraph after H3': [],
                'Top5 Link': [],
                'Top5 H1': [],
                'Top5 Paragraph after H1': [],
                'Top5 H2': [],
                'Top5 Paragraph after H2': [],
                'Top5 H3': [],
                'Top5 Paragraph after H3': [],
                'Top6 Link': [],
                'Top6 H1': [],
                'Top6 Paragraph after H1': [],
                'Top6 H2': [],
                'Top6 Paragraph after H2': [],
                'Top6 H3': [],
                'Top6 Paragraph after H3': [],
                'Top7 Link': [],
                'Top7 H1': [],
                'Top7 Paragraph after H1': [],
                'Top7 H2': [],
                'Top7 Paragraph after H2': [],
                'Top7 H3': [],
                'Top7 Paragraph after H3': [],
                'Top8 Link': [],
                'Top8 H1': [],
                'Top8 Paragraph after H1': [],
                'Top8 H2': [],
                'Top8 Paragraph after H2': [],
                'Top8 H3': [],
                'Top8 Paragraph after H3': [],
                'Top9 Link': [],
                'Top9 H1': [],
                'Top9 Paragraph after H1': [],
                'Top9 H2': [],
                'Top9 Paragraph after H2': [],
                'Top9 H3': [],
                'Top9 Paragraph after H3': [],
                'Top10 Link': [],
                'Top10 H1': [],
                'Top10 Paragraph after H1': [],
                'Top10 H2': [],
                'Top10 Paragraph after H2': [],
                'Top10 H3': [],
                'Top10 Paragraph after H3': [],
                'Relative Searches': []
            }
        
        results, relative_keywords=scrape_google(str(input_user.iloc[i,0]))
#         print(results)
        df = pd.DataFrame({'Links':results}) 
        df.to_csv('Links.csv', index=False)
        
        # detecting wordpress sites
        links = pd.read_csv('Links.csv')
        c1=len(links)
        i1=0

        wp=0
        wpsites=[]

        while i1<c1:
            try:
                response = scraper.get(str(links.iloc[i1,0]), timeout=60)
                response.raise_for_status() # if response is successfull, no exception will be raised
                bsh = BeautifulSoup(response.content, 'html.parser')
                marker=bsh.find(class_='entry-content')
                faulty_link = 'https://www.pupbox.com/training/when-do-puppies-lose-their-baby-teeth/'
                if "wp-content" in response.text and marker:
                    if str(links.iloc[i1,0]) != faulty_link:
                        wpsites.append(str(links.iloc[i1,0]))
                        wp=wp+1

                if wp>9:
                    break    
            except requests.exceptions.RequestException as e:
                xxx=1
                # print(e)

            except requests.exceptions.HTTPError as err:
                xxx=1
                # print(err)



            i1=i1+1
        df1 = pd.DataFrame(wpsites, columns=['WPsites']) 
        df1.to_csv('wpsites.csv', index=False)
        
        os.remove('Links.csv')
        H1=[]
        PH1_final=[]
        H2=[]
        PH2_final=[]
        H3=[]
        PH3_final=[]

        i2=0
        wps = pd.read_csv('wpsites.csv')
        c2=len(wps)

        while i2<c2:
            try:
                html = scraper.get(str(wps.iloc[i2,0]), headers=header)
                html.raise_for_status() # if response is successfull, no exception will be raised
                soup = BeautifulSoup(html.content, 'html.parser')
                content = soup.find('div', {'class', 'entry-content'})
                try:
                    h1 = soup.find('h1', {'class', 'entry-title'})
                except:
                    try:
                        h1 = soup.find('h1')
                    except:
                        h1 = content.find('h1')
                h1 = filter_html(str(h1))
                try:
                    p1 = filter_html(str(content.findAll('p')[0]))
                except:
                    try:
                        p1 = filter_html(str(content.find('p')))
                    except:
                        p1 = content.find('p').text
                H1.append(h1)
                PH1_final.append(p1)
                tags = []
                h2_index = 0
                p2_index = 0
                h3_index = 0
                p3_index = 0
                for u in content:
                    tags.append(str(u))
                for u in range(len(tags)):
                    if tags[u].startswith('<h2') or tags[u].startswith('<h3'):
                        h2 = filter_html(tags[u])
                        h2_index = u
                        H2.append(h2)
                        break
                for u in range(h2_index, len(tags)):
                    if tags[u].startswith('<p>'):
                        p2 = filter_html(tags[u])
                        p2_index = u
                        PH2_final.append(p2)
                        break
                for u in range(p2_index, len(tags)):
                    if tags[u].startswith('<h2') or tags[u].startswith('<h3'):
                        h3 = filter_html(tags[u])
                        h3_index = u
                        H3.append(h3)
                        break
                for u in range(h3_index, len(tags)):
                    if tags[u].startswith('<p>'):
                        p3 = filter_html(tags[u])
                        p3_index = u
                        PH3_final.append(p3)
                        break

                if len(H1)<i2+1:
                    H1.append(" ")
                if len(H2)<i2+1:
                    H2.append(" ")
                if len(H3)<i2+1:
                    H3.append(" ")  

                if len(PH1_final)<i2+1:
                    PH1_final.append(" ")    

                if len(PH2_final)<i2+1:
                    PH2_final.append(" ")

                if len(PH3_final)<i2+1:
                    PH3_final.append(" ")

            except requests.exceptions.RequestException as e:
                print(e)
                pass
            except requests.exceptions.HTTPError as err:
                print(err)
                pass
            except cloudscraper.exceptions.CloudflareChallengeError as error:
                print(error)
                pass
            
            i2=i2+1
        output_data['keyword'].append(input_user['Keyword'][i])
        for k in range(c2):
            output_data[f'Top{k+1} Link'].append(wps['WPsites'][k])
            output_data[f'Top{k+1} H1'].append(H1[k])
            output_data[f'Top{k+1} Paragraph after H1'].append(PH1_final[k])
            output_data[f'Top{k+1} H2'].append(H2[k])
            output_data[f'Top{k+1} Paragraph after H2'].append(PH2_final[k])
            output_data[f'Top{k+1} H3'].append(H3[k])
            output_data[f'Top{k+1} Paragraph after H3'].append(PH3_final[k])

        output_data['Relative Searches'].append(relative_keywords)
        
        os.remove('wpsites.csv')
        # loop to the next keyword
        
        if i==0:
            df_output = pd.DataFrame(output_data)
            df_output.to_csv('Output.csv', index=False)
        else:
            df_output = pd.DataFrame(output_data)
            df_output.to_csv('Output.csv', index=False, mode='a', header=False)
        i=i+1
        if UseDelay:
            time.sleep(60) # sleep 60 seconds
main()

In [ ]:
wps = pd.read_csv('wpsites.csv')
for i in range(len(wps)):
    print(wps['WPsites'][i])
input_user = pd.read_csv('keyword_example.csv')
for i in range(len(input_user)):
    print(input_user['Keyword'][i])

In [ ]:
def filter_html(string):
    string2 = ''
    for i in string:
        string2 +=i[0]
    check1 = 0
    check2 = 0
    while check1!=-1 and check2!=-1:
        if '<' and '>' in string2:
            end = string2.index('>')
            start = string2.index('<')
            string2 = string2.replace(string2[start:end+1], ' ')
            check1 = string2.find('<')
            check2 = string2.find('>')
            print(string2)
    return string2.strip()

In [ ]:

links = urls
c1=len(links)
i1=0

wp=0
wpsites=[]

while i1<c1:

    try:
        response = scraper.get(str(links[i1]), timeout=10)
        response.raise_for_status() # if response is successfull, no exception will be raised
        bsh = BeautifulSoup(response.content, 'html.parser')
        marker=bsh.find(class_='entry-content')

        if "wp-content" in response.text and marker:
            wpsites.append(links[i1])
            wp=wp+1

        if wp>9:
            break    
    except requests.exceptions.RequestException as e:
        xxx=1
        #print(e)

    except requests.exceptions.HTTPError as err:
        xxx=1
        #print(err)



    i1=i1+1

In [1]:
import requests
import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os
import re
import pandas as pd
import csv
import time
from requests_html import HTML
from requests_html import HTMLSession
from itertools import cycle
import cloudscraper

In [2]:
scraper = cloudscraper.create_scraper()
html = requests.get('https://www.caninejournal.com/how-to-housebreak-a-puppy/')
soup = BeautifulSoup(html.content, 'html.parser')

In [4]:
content = soup.select('[class*=entry-content]')
print(content)

None


In [ ]:
H1 = []
H1.append(x[0])

In [ ]:
marker=soup.find(class_='entry-content')
for line in marker:
    if str(line).startswith('<h') and h==3:
        h=4

        break

    if str(line).startswith('<p>') and h==3:

        temp = line.text

#         PH3.append(temp7)
        print(temp)

    if str(line).startswith('<h') and h==2:
        h=3
        temp = line.text
        print(temp)
#         H3.append(temp6)

    if str(line).startswith('<p>') and h==2:

        temp = line.text
        print(temp)       

    if str(line).startswith('<h') and h==1:
        h=2

        temp = line.text
        print(temp)

    if str(line).startswith('<p>') and h==1:

        temp = line.text
        print(temp)


In [ ]:
H1=[]
PH1_final=[]
H2=[]
PH2_final=[]
H3=[]
PH3_final=[]

i2=0
c2=len(wpsites)

while i2<c2:
    PH1=[]
    PH2=[]
    PH3=[]
    try:
        html = requests.get(wpsites[i2])
        html.raise_for_status() # if response is successfull, no exception will be raised
        bsh = BeautifulSoup(html.content, 'html.parser')


        #H1
        x=re.findall('<h.*>(.*)</h1>', str(bsh.h1))
        try:
            H1.append(x[0])
        except:
            H1.append(" ")

        marker=bsh.find(class_='entry-content')

        h=1

        # rest of H & P
        for line in marker:
            if str(line).startswith('<h') and h==3:
                h=4

                break

            if str(line).startswith('<p>') and h==3:

                temp = line.text

                PH3.append(temp)

            if str(line).startswith('<h') and h==2:
                h=3
                temp = line.text
                H3.append(temp)

            if str(line).startswith('<p>') and h==2:

                temp = line.text
                PH2.append(temp)       

            if str(line).startswith('<h') and h==1:
                h=2

                temp = line.text
                H2.append(temp)

            if str(line).startswith('<p>') and h==1:

                temp = line.text
                PH1.append(temp)
        PH1_final.append("\n".join(PH1))
        PH2_final.append("\n".join(PH2))
        PH3_final.append("\n".join(PH3))

        if len(H1)<i2+1:
            H1.append(" ")
        if len(H2)<i2+1:
            H2.append(" ")
        if len(H3)<i2+1:
            H3.append(" ")  

        if len(PH1_final)<i2+1:
            PH1_final.append(" ")    

        if len(PH2_final)<i2+1:
            PH2_final.append(" ")

        if len(PH3_final)<i2+1:
            PH3_final.append(" ")

    except requests.exceptions.RequestException as e:
        print(e)

    except requests.exceptions.HTTPError as err:
        print(err)
    i2=i2+1

        PH1_final.append("\n".join(PH1))
        PH2_final.append("\n".join(PH2))
        PH3_final.append("\n".join(PH3))

        if len(H1)<i2+1:
            H1.append(" ")
        if len(H2)<i2+1:
            H2.append(" ")
        if len(H3)<i2+1:
            H3.append(" ")  

        if len(PH1_final)<i2+1:
            PH1_final.append(" ")    

        if len(PH2_final)<i2+1:
            PH2_final.append(" ")

        if len(PH3_final)<i2+1:
            PH3_final.append(" ")
    


try:
    output_data.append([str(input_user.iloc[i,0]),H1[0],PH1_final[0],H2[0],PH2_final[0],H3[0],PH3_final[0],H1[1],PH1_final[1],H2[1],PH2_final[1],H3[1],PH3_final[1],H1[2],PH1_final[2],H2[2],PH2_final[2],H3[2],PH3_final[2],H1[3],PH1_final[3],H2[3],PH2_final[3],H3[3],PH3_final[3],H1[4],PH1_final[4],H2[4],PH2_final[4],H3[4],PH3_final[4],H1[5],PH1_final[5],H2[5],PH2_final[5],H3[5],PH3_final[5],H1[6],PH1_final[6],H2[6],PH2_final[6],H3[6],PH3_final[6],H1[7],PH1_final[7],H2[7],PH2_final[7],H3[7],PH3_final[7],H1[8],PH1_final[8],H2[8],PH2_final[8],H3[8],PH3_final[8],H1[9],PH1_final[9],H2[9],PH2_final[9],H3[9],PH3_final[9]])
except:
    output_data.append([str(input_user.iloc[i,0])," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "])

    


In [ ]:
for line in marker:
    if (str(line).startswith('<p')):
        print(line.text)
        break

In [ ]:
for line in marker:
    print(str(line))

In [ ]:
wpsites[-1]

In [ ]:
input_user = pd.read_csv('keyword_example.csv')
c = len(input_user)
c

In [ ]:
string = '<a href="https://pupbox.com/training/pupbox-picks-teething-product-guide/">here</a>'
start = string.index('<')
ends = string.index('>')
string = string[ends+1:]

In [ ]:
string

In [ ]:
string.find('c')

In [ ]:
def filter_html(string):
    string2 = ''
    for i in string:
        string2 +=i[0]
    check1 = 0
    check2 = 0
    while check1!=-1 and check2!=-1:
        end = string2.index('>')
        start = string2.index('<')
        string2 = string2.replace(string2[start:end+1], ' ')
        check1 = string2.find('<')
        check2 = string2.find('>')
    return string2.strip()

In [ ]:
string = [['<a href="https://pupbox.com/training/pupbox-picks-teething-product-guide/">here</a>'],['<img alt="When Do Puppies Lose Their Baby Teeth" class="alignnone size-full wp-image-392323" height="555" loading="lazy" sizes="(max-width: 1024px) 100vw, 1024px" src="https://pupbox.com/wp-content/uploads/2019/07/When-Do-Puppies-Lose-Their-Baby-Teeth_.jpg" srcset="https://www.pupbox.com/wp-content/uploads/2019/07/When-Do-Puppies-Lose-Their-Baby-Teeth_.jpg 1024w, https://www.pupbox.com/wp-content/uploads/2019/07/When-Do-Puppies-Lose-Their-Baby-Teeth_-701x380.jpg 701w, https://www.pupbox.com/wp-content/uploads/2019/07/When-Do-Puppies-Lose-Their-Baby-Teeth_-768x416.jpg 768w, https://www.pupbox.com/wp-content/uploads/2019/07/When-Do-Puppies-Lose-Their-Baby-Teeth_-380x206.jpg 380w" width="1024"/></p><p>Just like human babies, animals have baby teeth that need to be replaced as they get older. This is true of puppies; it is a natural part of life, but it makes things no less scary when you notice your own pup losing his at random.</p>']]
filter_html(string)

In [ ]:
end = string.index('>')
start = string.index('<')
string.replace(string[start:end+1], ' ')
check1 = string.find('<')
check2 = string.find('>')
print(string)

In [ ]:
string[start:end+1]

In [ ]:
string.replace(string[start:end+1], ' ')

In [ ]:
string = '<img alt="When Do Puppies Lose Their Baby Teeth" class="alignnone size-full wp-image-392323" height="555" loading="lazy" sizes="(max-width: 1024px) 100vw, 1024px" src="https://pupbox.com/wp-content/uploads/2019/07/When-Do-Puppies-Lose-Their-Baby-Teeth_.jpg" srcset="https://www.pupbox.com/wp-content/uploads/2019/07/When-Do-Puppies-Lose-Their-Baby-Teeth_.jpg 1024w, https://www.pupbox.com/wp-content/uploads/2019/07/When-Do-Puppies-Lose-Their-Baby-Teeth_-701x380.jpg 701w, https://www.pupbox.com/wp-content/uploads/2019/07/When-Do-Puppies-Lose-Their-Baby-Teeth_-768x416.jpg 768w, https://www.pupbox.com/wp-content/uploads/2019/07/When-Do-Puppies-Lose-Their-Baby-Teeth_-380x206.jpg 380w" width="1024"/></p><p>Just like human babies, animals have baby teeth that need to be replaced as they get older. This is true of puppies; it is a natural part of life, but it makes things no less scary when you notice your own pup losing his at random.</p>'

In [ ]:
import requests
import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os
import re
import pandas as pd
import time
from requests_html import HTML
from requests_html import HTMLSession
from itertools import cycle
import cloudscraper
scraper = cloudscraper.create_scraper()


In [ ]:
UseProxy = False
def get_source(url):
    
    if UseProxy:
        done=False
        while done==False:
            
            proxy_pool = cycle(proxies)
            for p in range(1,11):
                #Get a proxy from the pool
                proxy = next(proxy_pool)
                
                try:
                    session = requests.session()
                    session = cloudscraper.create_scraper(sess = session)
                    session.proxies = {
                                "http": proxy,
                                "https": proxy,
                                }
                    response = session.get(url, headers=header)
                    response.raise_for_status() # if response is successfull, no exception will be raised
                    done=True
                    
                    return response
                    
                
                except requests.exceptions.RequestException as e:
                    print(e)
                    print(" ")
                    print("retrying with new proxy")
                    
                except requests.exceptions.HTTPError as err:
                    print(err)
                    print(" ")
                    print("retrying with new proxy")    
    else:
        try:
            session = requests.session()
            session = cloudscraper.create_scraper(sess = session)
            session.headers['user-agent'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.81 Safari/537.36'
            response = session.get(url)
            response.raise_for_status() # if response is successfull, no exception will be raised   
            return response
                            
        except requests.exceptions.RequestException as e:
            print(e)
                                
        except requests.exceptions.HTTPError as err:
            print(err)
            
        except cloudscraper.exceptions.CloudflareChallengeError as error:
            print(error)
            pass
            
def scrape_google(query):

     
    query = urllib.parse.quote_plus(query)
    
   
    response = get_source("https://www.google.com/search?q=" + query + "&num=100")
    soup = BeautifulSoup(response.content, 'html.parser')
    
    relative_keywords = soup.find('div', {'çlass':'y6Uyqe'})
    decks = soup.findAll('div', {'class':'AJLUJb'})
    searches = []
    for deck in decks:
        cards = deck.findAll('div')
        for card in cards:
            if card.text != '':
                searches.append(card.text)
    cards = soup.findAll('div', {'class' : 'yuRUbf'})
    links = []
    for card in cards:
        link = card.find('a')['href']
        links.append(link)
        
    google_domains = ('https://www.google.', 
                      'https://google.', 
                      'https://webcache.googleusercontent.', 
                      'http://webcache.googleusercontent.', 
                      'https://policies.google.',
                      'https://support.google.',
                      'https://maps.google.',
                      'https://translate.google.',
                      'https://www.youtube')

    for url in links[:]:
        
        if url.startswith(google_domains):
            links.remove(url)
    
    
    return links, searches

In [ ]:
output_data=[]
input_user = pd.read_csv('keyword_example.csv')
c = len(input_user)
i=0
while i<1:

    results, relative_keywords=scrape_google(str(input_user.iloc[i,0]))
#         print(results)
    df = pd.DataFrame({'Links':results}) 
    df.to_csv('Links.csv', index=False)

    # detecting wordpress sites
    links = pd.read_csv('Links.csv')
    c1=len(links)
    i1=0

    wp=0
    wpsites=[]

    while i1<c1:
        try:
            response = scraper.get(str(links.iloc[i1,0]), timeout=60)
            response.raise_for_status() # if response is successfull, no exception will be raised
            bsh = BeautifulSoup(response.content, 'html.parser')
            marker=bsh.find(class_='entry-content')

            if "wp-content" in response.text and marker:
                wpsites.append(str(links.iloc[i1,0]))
                wp=wp+1

            if wp>9:
                break    
        except requests.exceptions.RequestException as e:
            xxx=1
            # print(e)

        except requests.exceptions.HTTPError as err:
            xxx=1
            # print(err)



        i1=i1+1
    df1 = pd.DataFrame(wpsites, columns=['WPsites']) 
    print(wpsites)

In [1]:
wpsites = ['https://www.puppyleaks.com/potty-training-your-puppy/', 'https://puppyintraining.com/how-to-potty-train-a-puppy-in-an-apartment/', 'https://www.rover.com/blog/complete-guide-puppy-potty-training/', 'https://kaufmannspuppytraining.com/en/how-to-potty-train-a-puppy-fast/', 'https://thezebra.org/2018/10/01/potty-training-101-housetraining-your-new-puppy-in-two-weeks-or-less/', 'https://drsophiayin.com/blog/entry/a_foolproof_potty-training_plan/', 'https://cherishpetfood.com.au/tag/can-an-8-week-old-puppy-be-potty-trained/', 'https://journeydogtraining.com/how-to-potty-train-a-puppy-fast/', 'https://www.dogtrainingnow.com/2018/02/common-potty-training-mistakes/', 'https://www.caninejournal.com/how-to-housebreak-a-puppy/']

In [ ]:
sess = requests.session()
scraper = cloudscraper.create_scraper(sess)
header = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.81 Safari/537.36'
}

c2 = len(wpsites)
for i in wpsites:
    print(i)
    html = scraper.get(str(i), headers=header)
    html.raise_for_status() # if response is successfull, no exception will be raised
    soup = BeautifulSoup(html.content, 'html.parser')
    content = soup.find('div', {'class', 'entry-content'})

    try:
        h1 = soup.find('h1', {'class', 'entry-title'}).text.strip()
    except:
        h1 = soup.find('h1').text.strip()
    
    p1 = content.find('p').text.strip()
    if p1 == '':
        p1 = content.findAll('p')[1].text
    print(h1)
    print(p1)
    tags = []
    h2_index = 0
    p2_index = 0
    h3_index = 0
    p3_index = 0
    for i in content:
        tags.append(str(i))
    for i in range(len(tags)):
        if tags[i].startswith('<h2') or tags[i].startswith('<h3'):
            h2 = filter_html(tags[i])
            h2_index = i
            print(h2)
            break
    for i in range(h2_index, len(tags)):
        if tags[i].startswith('<p>'):
            p2 = filter_html(tags[i])
            p2_index = i
            print(p2)
            break
    for i in range(p2_index, len(tags)):
        if tags[i].startswith('<h2') or tags[i].startswith('<h3'):
            h3 = filter_html(tags[i])
            h3_index = i
            print(h3)
            break
    for i in range(h3_index, len(tags)):
        if tags[i].startswith('<p>'):
            p3 = filter_html(tags[i])
            p3_index = i
            print(p3)
            break

In [ ]:
wpsites

In [ ]:
tags = []
h2_index = 0
p2_index = 0
h3_index = 0
p3_index = 0
for i in content:
    tags.append(str(i))
for i in range(len(tags)):
    if tags[i].startswith('<h2') or tags[i].startswith('<h3'):
        h2 = filter_html(tags[i])
        h2_index = i
        print(h2)
        break
for i in range(h2_index, len(tags)):
    if tags[i].startswith('<p>'):
        p2 = filter_html(tags[i])
        p2_index = i
        print(p2)
        break
for i in range(p2_index, len(tags)):
    if tags[i].startswith('<h2') or tags[i].startswith('<h3'):
        h3 = filter_html(tags[i])
        h3_index = i
        print(h3)
        break
for i in range(h3_index, len(tags)):
    if tags[i].startswith('<p>'):
        p3 = filter_html(tags[i])
        p3_index = i
        print(p3)
        break

In [ ]:
output_data = [(input_user['Keyword'][i]), wps['WPsites'][0],H1[0],PH1_final[0],H2[0],PH2_final[0],H3[0],PH3_final[0], wps['WPsites'][1],H1[1],PH1_final[1],H2[1],PH2_final[1],H3[1],PH3_final[1], wps['WPsites'][2],H1[2],PH1_final[2],H2[2],PH2_final[2],H3[2],PH3_final[2], wps['WPsites'][3],H1[3],PH1_final[3],H2[3],PH2_final[3],H3[3],PH3_final[3], wps['WPsites'][4],H1[4],PH1_final[4],H2[4],PH2_final[4],H3[4],PH3_final[4], wps['WPsites'][5],H1[5],PH1_final[5],H2[5],PH2_final[5],H3[5],PH3_final[5], wps['WPsites'][6],H1[6],PH1_final[6],H2[6],PH2_final[6],H3[6],PH3_final[6], wps['WPsites'][7],H1[7],PH1_final[7],H2[7],PH2_final[7],H3[7],PH3_final[7],wps['WPsites'][8],H1[8],PH1_final[8],H2[8],PH2_final[8],H3[8],PH3_final[8],wps['WPsites'][9],H1[9],PH1_final[9],H2[9],PH2_final[9],H3[9],PH3_final[9],relative_keywords]

In [ ]:
header = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.81 Safari/537.36'
}

html = scraper.get('https://www.puppyleaks.com/potty-training-your-puppy/', headers=header)
html.raise_for_status() # if response is successfull, no exception will be raised
soup = BeautifulSoup(html.content, 'html.parser')
content = soup.find('div', {'class', 'entry-content'})
h1 = content.findAll('p')
h1 = h1[0].text + h1[1].text
print(h1)


In [2]:
proxies = ['154.17.87.216:29842:kgrzes:wKcGrD43', '154.17.87.128:29842:kgrzes:wKcGrD43', '154.17.87.149:29842:kgrzes:wKcGrD43', '154.17.87.221:29842:kgrzes:wKcGrD43', '154.17.87.113:29842:kgrzes:wKcGrD43']
def filter_string(string):
    if '[' or ']' in string:
        string = string.replace('[', ' ')
        string = string.replace(']', ' ')
    string = string.replace("'", " ")
    string.replace('  ', ' ')
    string.replace('   ', ' ')
    string = string.strip()
    return string
def filter_html(string):
    check1 = 0
    check2 = 0
    while check1!=-1 and check2!=-1:
        if '<' and '>' in string:
            end = string.index('>')
            start = string.index('<')
            string = string.replace(string[start:end+1], ' ')
            check1 = string.find('<')
            check2 = string.find('>')
        else:
            break
    return string.strip()

In [6]:
html = scraper.get(wpsites[0], headers=header)
html.raise_for_status() # if response is successfull, no exception will be raised
soup = BeautifulSoup(html.content, 'html.parser')
content = soup.find('div', {'class', 'entry-content'})
if content == None:
    content = soup.select('[class*=entry-content]')[0]
try:
    h1 = soup.find('h1', {'class', 'entry-title'})
except:
    try:
        h1 = soup.find('h1')
    except:
        h1 = content.find('h1')
h1 = filter_html(str(h1))
try:
    full_p1 = ''
    p1 = content.findAll('p')[0:2]
    for para in p1:
        p1 = filter_html(str(para))
        full_p1+=p1
except:
    try:
        p1 = filter_html(str(content.find('p')))
    except:
        try:  
            p1 = content.find('p').text
        except:
            try:
                p1 = soup.find('p').text
            except:
                pass
print(h1)
print(full_p1)
tags = []
h2_index = 0
p2_index = 0
h3_index = 0
p3_index = 0
for u in content:
    tags.append(str(u))
for u in range(len(tags)):
    if tags[u].startswith('<h2') or tags[u].startswith('<h3'):
        h2 = filter_html(tags[u])
        h2_index = u
        print(h2)
        break
for u in range(h2_index+1, len(tags)):
    full_p2 = ''
    if tags[u].startswith('<h2') or tags[u].startswith('<h3'):
        break
    if tags[u].startswith('<p>'):
        p2 = filter_html(tags[u])
        full_p2 += p2
        p2_index = u
        print(full_p2)
for u in range(p2_index, len(tags)):
    if tags[u].startswith('<h2') or tags[u].startswith('<h3'):
        h3 = filter_html(tags[u])
        h3_index = u
        print(h3)
        break
for u in range(h3_index+1, len(tags)):
    full_p3 = ''
    if tags[u].startswith('<h2') or tags[u].startswith('<h3'):
        break
    if tags[u].startswith('<p>'):
        p3 = filter_html(tags[u])
        full_p3+=p3
        p3_index = u
        print(full_p3)

5 Simple Tips For Potty Training Your Puppy
Last updated on  December 13, 2021  By   Puppy Leaks     15 CommentsIn the late 90’s I found a rather simple way to potty train a puppy, and it worked so well I’ve been using it ever since. After a week of using these tips you’ll eliminate 90% of accidents, and you’ll be well on the way to a having a fully house trained pup.
1. Pay Attention to Your Puppy At All Times
A quiet puppy is trouble, or so the saying goes. Whether he’s getting into the garbage, eating your new shoes or pooping over in the corner — a quiet puppy signals trouble.
If you want to prevent accidents before they happen you’re going to need to watch your pup at all times, including every time they wander off. It only takes one accident to set your training back. Now I know that watching your puppy non-stop isn’t exactly fun &amp; exciting, but being able to catch them before they have an accident is why this method works so well.
If you’re like me and have trouble keeping u

In [4]:
import requests
import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os
import re
import pandas as pd
import csv
import time
from requests_html import HTML
from requests_html import HTMLSession
from itertools import cycle
import cloudscraper

sess = requests.session()
scraper = cloudscraper.create_scraper(sess)
header = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.81 Safari/537.36'
}